# Hartree-Fock Analytic Nuclear Gradients Implementation

This notebook demonstrates how to compute analytic nuclear gradients for Hartree-Fock theory using Psi4Numpy.
The implementation follows the equations from Szabo and Ostlund's "Modern Quantum Chemistry".

## Theory Overview
For a closed-shell system, the Hartree-Fock energy gradient with respect to nuclear coordinate x is:

            # add code to contract eri_derivs with D to get J_deriv. J_uv = 2 * sum_ls (uv|ls) D_ls 
            J_deriv[deriv_index] = 2 * np.einsum("uvls,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract eri_derivs with D to get K_deriv. K_uv = -1 * sum_ls (ul|vs) D_ls
            K_deriv[deriv_index] = -1 * np.einsum("ulvs,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

$$\frac{\partial E_{HF}}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} + 2 \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i} - \sum_{\mu \nu \lambda \sigma} D_{\mu \nu} D_{\lambda \sigma} \frac{(\mu \lambda | \nu \sigma)}{\partial x_i}- \sum_{\mu\nu} W_{\mu\nu} \frac{\partial S_{\mu\nu}}{\partial x_i}$$

equivalently,

$$ \frac{\partial E_{HF}}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} +  \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i} - \frac{1}{2}\sum_{\mu \nu \lambda \sigma} D_{\mu \nu} D_{\lambda \sigma} \frac{(\mu \lambda | \nu \sigma)}{\partial x_i}- \sum_{pq} F_{pq} \frac{\partial S_{pq}}{\partial x_i} $$
and similarly for the $y$ and $z$ coordinates.  Importantly, each atom has $x$, $y$, and $z$ coordinates so this yields a gradient
vector with $3 N_{\rm atoms}$ elements.

The nuclear gradients of QED-HF just involve two additional terms:

$$ \frac{\partial E_{QED-HF}}{\partial x_i} =  \frac{\partial E_{HF}}{\partial x_i} - \frac{1}{2} \sum_{\mu \nu} D_{\mu \nu} \frac{\partial q_{\mu \nu}}{\partial x_i} - \sum_{\mu \nu} D_{\mu \nu} \sum_{\lambda \sigma} D_{\lambda \sigma} \frac{\partial d_{\mu \sigma}}{\partial x_i} d_{\lambda \nu} $$

**Two Important notes!**
1. The density matrix and Fock matrix must arise from QED-HF, not from HF
2. The derivatives of quadrupole integrals are not available from the MintsHelper class, so we can use the derivatives of the quadrupole moments intead

Where:
- $E_{nuc}$ is the nuclear repulsion energy
- $D_{\mu\nu}$ is the density matrix 
- $h_{\mu\nu}$ is the one-electron integral matrix in the AO basis
- $(\mu\nu|\lambda\sigma)$ are the two-electron integrals in the AO basis
- $S_{\mu\nu}$ is the overlap matrix in the AO basis
- $W_{\mu\nu}$ is the energy-weighted density matrix
- $F_{p q}$ is the Fock matrix in the MO basis
- $S_{pq}$ is the overlap matrix in the MO basis
- $d_{\mu \nu}$ are field-scaled dipole integrals in the AO basis
- $q_{\mu \nu}$ are field-scaled quadrupole integrals in the AO basis

# To-do items to adapt to QED-HF Gradients
- Add quadrupole and dipolar terms to Fock matrix in the function `def compute_fock_matrix_term(geometry_string, basis_set='sto-3g', method='scf', qed=False, lambda_list=[0, 0, 0]):`
- Add dipolar gradient terms to the function `def compute_two_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf', qed=False, lambda_list=[0, 0, 0]):`
- Add code to compute the quadrupole gradient, which could go in any method that computes the density matrix

Note: 
- For dipole integral derivatives, we want to use the MintsHelper function ao_elec_dip_deriv1(self: psi4.core.MintsHelper, atom: int) → List[psi4.core.Matrix] Gradient of AO basis electric dipole integrals: returns (3 * natoms) matrices
- FOr quadrupole derivatives, we want to use the MintsHelper function multipole_grad(self: psi4.core.MintsHelper, D: psi4.core.Matrix, order: int, origin: List[float]) → psi4.core.Matrix¶




# Numerical Hartree-Fock gradients or QED Hartree-Fock gradients
The Hartree-Fock energy gradient may also be approximated using centered finite differences as follows:
$$\frac{\partial E}{\partial x_i} \approx \frac{ E(x_1, ..., x_i + \delta x, x_{i+1}, ... x_N) - E(x_1, ..., x_i - \delta x, x_{i+1}, ... x_N) }{2 \delta x} $$
where $\delta x$ is a sufficiently small step along the nuclear coordinates.  

**Importantly** Numerical gradients in this approximation require two full solutions to the RHF energy for each nuclear degree of freedom, so in total $6 N_{atoms}$ RHF calculations are needed for each numerical gradient evaluation.  The analytical gradients require only a single solution of the RHF energy and then a series of contractions using the existing RHF density matrix.  

Nevertheless, we can set up a cell to compute the numerical gradient as a reference.

In [3]:
# First import the necessary libraries
import psi4
import numpy as np
np.set_printoptions(precision=15, linewidth=200, suppress=True)
from helper_CQED_RHF import *

In [4]:
# Expected gradient contributions for Water in a STO-3G basis set with the following geometry:
#h2o_string =
#    O      0.000000000000   0.000000000000   0.000000000000
#    H      0.000000000000   0.757000000000   0.587000000000
#    H      0.000000000000  -0.757000000000   0.587000000000
#no_reorient
#no_com
#symmetry c1


_expected_nuclear_gradient = np.array([
    [0.00000000000000,  0.00000000000000,  2.99204046891092],
    [0.00000000000000, -2.05144597283373, -1.49602023445546],
    [0.00000000000000,  2.05144597283373, -1.49602023445546],
])

_overlap_gradient = np.array([
    [-0.00000000000000, -0.00000000000000,  0.30728746121587],
    [ 0.00000000000000, -0.14977126575800, -0.15364373060793],
    [-0.00000000000000,  0.14977126575800, -0.15364373060793],
])

_potential_gradient = np.array([
    [-0.00000000000000,  0.00000000000002, -6.81982772856799],
    [-0.00000000000000,  4.38321774316664,  3.40991386428399],
    [ 0.00000000000000, -4.38321774316666,  3.40991386428400],
])
_kinetic_gradient = np.array([
    [ 0.00000000000000, -0.00000000000000,  0.66968290617933],
    [ 0.00000000000000, -0.43735698924315, -0.33484145308966],
    [-0.00000000000000,  0.43735698924315, -0.33484145308967],
])

_coulomb_gradient = np.array([
    [ 0.00000000000000, -0.00000000000002,  3.34742251141627],
    [ 0.00000000000000, -2.03756324433539, -1.67371125570813],
    [-0.00000000000000,  2.03756324433541, -1.67371125570814],
])

_exchange_gradient = np.array([
    [-0.00000000000000,  0.00000000000000, -0.43559674130726],
    [-0.00000000000000,  0.26932748463493,  0.21779837065363],
    [ 0.00000000000000, -0.26932748463493,  0.21779837065363],
])

# numerical QED-RHF gradient for H2O in a sto-3g basis set with lambda = [0, 0, 0.05]
_expected_qed_rhf_gradient = np.array([
    [ 0.00000000e+00,  0.00000000e+00,  5.85242485e-02],
    [-3.76002252e-12, -2.33369805e-02, -2.92621343e-02],
    [ 7.52004504e-12,  2.33369805e-02, -2.92621343e-02],
])

In [5]:
def modify_geometry_string(geometry_string, displacement_array):
    """
    Extracts Cartesian coordinates from a Psi4 geometry string, applies a
    transformation function to the coordinates, and returns a new geometry string.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        transformation_function (callable): A function that takes a NumPy
            array of Cartesian coordinates (N x 3) as input and returns a
            NumPy array of the same shape with the transformed coordinates.

    Returns:
        str: A new Psi4 molecular geometry string with the transformed coordinates.
    """
    lines = geometry_string.strip().split('\n')
    atom_data = []
    symmetry = None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.lower().startswith("symmetry"):
            symmetry = line
            continue
        parts = line.split()
        if len(parts) == 4:
            atom = parts[0]
            try:
                x, y, z = map(float, parts[1:])
                atom_data.append([atom, x, y, z])
            except ValueError:
                # Handle cases where the line might not be atom coordinates
                pass

    if not atom_data:
        return ""

    coordinates = np.array([[data[1], data[2], data[3]] for data in atom_data])

    # Apply the transformation function
    transformed_coordinates = displacement_array  + coordinates

    new_geometry_lines = []
    for i, data in enumerate(atom_data):
        atom = data[0]
        new_geometry_lines.append(f"{atom} {transformed_coordinates[i, 0]:.8f} {transformed_coordinates[i, 1]:.8f} {transformed_coordinates[i, 2]:.8f}")

    new_geometry_string = "\n".join(new_geometry_lines)
    if symmetry:
        new_geometry_string += f"\n{symmetry}"

    return f"""{new_geometry_string}"""


def run_psi4_calculation(geometry_string, displacement_array, basis_set='sto-3g', method='scf'):
    """
    Runs a Psi4 calculation with the given geometry string, basis set, and method.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.
        

    Returns:
        dict: A dictionary containing the results of the Psi4 calculation.
    """
    # Modify the geometry string with the displacement array
    modified_geometry_string = modify_geometry_string(geometry_string, displacement_array)

    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(modified_geometry_string)
        energy = psi4.energy(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return energy

def run_qedhf_calculation(geometry_string, displacement_array, lambda_list = [0, 0, 0], basis_set='sto-3g'):
    """
    Runs a QED-RHF calculation with a given geometry, displacement, lambda vector, and basis set

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        lambda_list (list): A list of lambda values for the calculation, defaults to [0, 0, 0].
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.
        

    Returns:
        dict: A dictionary containing the results of the Psi4 calculation.
    """
    # Modify the geometry string with the displacement array
    test_str = modify_geometry_string(geometry_string, displacement_array)
    print("Just entered run_qedhf_calculation")
    print("Modified geometry string:")
    print(test_str)

    # save options to ditionary
    options_dict = {
        'basis' : basis_set,
        'scf_type' : 'pk',
        'e_convergence' : 1e-12,
        'd_convergence' : 1e-12,
        'save_jk' : True,
    }

    # Set up Psi4 options
    psi4.set_options(options_dict)

    lambda_vector = np.array(lambda_list)

    # Run the Psi4 calculation
    pqed_dict = cqed_rhf(lambda_vector, test_str, options_dict)

    return pqed_dict


def compute_cqed_numerical_gradient(geometry_string, lambda_list=[0, 0, 0], basis_set='sto-3g', displacement_unit=0.001):
    """
    Computes the numerical gradient of the CQED-RHF energy with respect to the geometry.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        lambda_list (list): A list of lambda values for the calculation, defaults to [0, 0, 0].
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        displacement_unit (float): The step size for finite difference calculations, defaults to 0.001

    Returns:
        np.ndarray: The numerical gradient of the CQED-RHF energy with respect to the geometry.
    """
    # Get the number of atoms from the geometry string
    num_atoms = len(geometry_string.strip().split('\n')) - 3
    print("Number of atoms:", num_atoms)
    # Initialize the numerical gradient array
    numerical_gradient = np.zeros((num_atoms, 3))
    nuc_dip_gradient = np.zeros((3 * num_atoms, 3))
    rhf_dip_gradient = np.zeros((3 *num_atoms, 3))
    print("Initialized rhf dip gradient array:\n")
    print(rhf_dip_gradient)
    
    displacement_array = np.zeros((num_atoms, 3))
    print("Displacement array:\n")
    print(displacement_array)
    print("modify_geometry_string\n")
    print(geometry_string)

    # Compute the energy at the original geometry
    original_energy = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)["CQED-RHF ENERGY"]

    # Compute the energy at each perturbed geometry
    for i in range(num_atoms):

        for j in range(3):

            deriv_index = 3 * i + j

            displacement_array = np.zeros((num_atoms, 3))
            displacement_array[i, j] = displacement_unit
            
            # print the displacement unit vector
            print("!!!! DISPLACEMENT UNIT VECTOR:\n")
            print(displacement_array)

            # Insert code to compute psi4 energy at forward displacement
            ret_f = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)
            displacement_array *= -1
            ret_b = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)

            e_f = ret_f["CQED-RHF ENERGY"]
            e_b = ret_b["CQED-RHF ENERGY"]

            # Insert code to compute finite difference along this displacement
            grad_element = (e_f - e_b) / (2 * 1.88973 * displacement_unit)
            numerical_gradient[i, j] = grad_element

    return original_energy, numerical_gradient 


def compute_psi4_analytical_gradient(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Computes the energy gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The energy gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(geometry_string)
        gradient = psi4.gradient(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return np.asarray(gradient)

def compute_psi4_numerical_gradient(geometry_string, displacement_unit=0.01, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Computes the numerical gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_unit (float): The unit of displacement for numerical gradient calculation, defaults to 0.01.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The numerical gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Get the number of atoms from the geometry string
    num_atoms = len(geometry_string.strip().split('\n')) - 3

    # Initialize the numerical gradient array
    numerical_gradient = np.zeros((num_atoms, 3))

    # loop over all atoms
    for i in range(num_atoms):

        # loop over all three dimensions
        for j in range(3):
            # Create a displacement unit vector
            displacement_array = np.zeros((num_atoms, 3))
            displacement_array[i, j] = displacement_unit
            
            # print the displacement unit vector
            print(displacement_array)

            # Insert code to compute psi4 energy at forward displacement
            e_f = run_psi4_calculation(geometry_string, displacement_array, basis_set, method)

            # Insert code to compute psi4 energy at backwards displacement
            displacement_array *= -1
            e_b = run_psi4_calculation(geometry_string, displacement_array, basis_set, method)

            # Insert code to compute finite difference along this displacement
            grad_element = (e_f - e_b) / (2 * 1.88973 * displacement_unit)

            # Insert code to store this to the appropriate gradient element
            numerical_gradient[i, j] = grad_element


    # return the gradient
    return numerical_gradient 


 


In [6]:
starting_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
no_reorient
no_com
symmetry c1
"""

starting_displacement = np.array([[0, 0.0, 0.0], [0, 0, 0,], [0, 0, .0]])

# Run the Psi4 calculation
energy = run_psi4_calculation(starting_string, starting_displacement, basis_set='sto-3g', method='scf')
gradient = compute_psi4_analytical_gradient(starting_string, basis_set='sto-3g', method='scf')

# test the numerical gradient
numerical_gradient = compute_psi4_numerical_gradient(starting_string, displacement_unit=0.01, basis_set='sto-3g', method='scf')


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Fri May 30 13:33:50 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------

In [7]:
print("Energy:", energy)
print("Gradient:\n", gradient)
print("Numerical Gradient:\n", numerical_gradient)

# compute error between numerical and analytical gradient
error = numerical_gradient - gradient
print("Error:\n", error)
print("Norm of error:\n",np.linalg.norm(error))

Energy: -74.96306312979318
Gradient:
 [[ 0.                 0.000000000000001  0.061008877849039]
 [ 0.                -0.023592244369035 -0.030504438924519]
 [-0.                 0.023592244369034 -0.03050443892452 ]]
Numerical Gradient:
 [[ 0.                -0.000000000000752  0.061016771477982]
 [ 0.000000000000752 -0.023644674278839 -0.030509434267519]
 [ 0.                 0.023644674272823 -0.030509434272031]]
Error:
 [[-0.                -0.000000000000753  0.000007893628944]
 [ 0.000000000000752 -0.000052429909805 -0.000004995343   ]
 [ 0.                 0.000052429903789 -0.000004995347511]]
Norm of error:
 7.489997716813086e-05


# Function to compute nuclear gradient
$$\frac{\partial E_{nuc}}{\partial x_i}$$ 


In [8]:
def compute_nuclear_repulsion_gradient(geometry_string):
    """
    Method to compute the nuclear repulsion gradient

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    The nuclear repulsion gradient only depends on the atom identities and positions
    """
    # Define your molecular geometry
    molecule = psi4.geometry(geometry_string)

    # get the nuclear repulsion gradient
    nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())
    
    return nuclear_repulsion_gradient

# Compute the nuclear repulsion gradient using the starting_string
nuclear_repulsion_gradient = compute_nuclear_repulsion_gradient(starting_string)
print("Nuclear Repulsion Gradient:\n", nuclear_repulsion_gradient)


# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(nuclear_repulsion_gradient, _expected_nuclear_gradient):
    print("Nuclear repulsion gradient is correct.")
else:
    print("Nuclear repulsion gradient is incorrect.")
    print("Computed:\n", nuclear_repulsion_gradient)
    print("Expected:\n", _expected_nuclear_gradient)

Nuclear Repulsion Gradient:
 [[ 0.                 0.                 2.992040468910919]
 [ 0.                -2.051445972833726 -1.49602023445546 ]
 [ 0.                 2.051445972833726 -1.49602023445546 ]]
Nuclear repulsion gradient is correct.


# Function to compute Fock matrix expressed in the MO basis
$$ F_{ij} = V_{ij} + T_{ij} + 2 \sum_{k}^{occ} (ii|kk) - \sum_{k}^{occ} (ik|kj) $$

If we have QED terms, we will need to add the following:

$$  F^{QED}_{ij} = F_{ij} - \frac{1}{2} q_{ij} - \sum_{k}^{occ} d_{ik} d_{kj} $$

**Note: We need to add these QED terms now!**


In [9]:
def compute_fock_matrix_term(geometry_string, basis_set='sto-3g', method='scf', qed=False, lambda_list=[0, 0, 0]):
    """
    Method to compute the Fock matrix

    Arguments
    ---------

    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The Fock matrix is the matrix representation of the Fock operator, which is used in Hartree-Fock calculations.
    To compute the Fock matrix in the MO basis, we need the one-electron and two-electron integrals and the density matrix.
    We will get these from a converged Hartree-Fock calculation.
    """

    # set up the molecule
    molecule = psi4.geometry(geometry_string)

    # Set up Psi4 options
    options_dict = {
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    }
    psi4.set_options(options_dict)

    # set up the geometry
    psi4.geometry(geometry_string)
    
    if qed:
        pqed_dict = cqed_rhf(lambda_list, geometry_string, options_dict)
        rhf_e = pqed_dict["CQED-RHF ENERGY"]
        wfn = pqed_dict["CQED-RHF WFN"]
        
        # get the number of orbitals and the number of doubly occupied orbitals
        n_orbitals = wfn.nmo()
        n_docc = wfn.nalpha()

        # Get QED-Fock matrix from pqed_dict
        F = pqed_dict["CQED-RHF FOCK"]
        
        # instantiate the MintsHelper object
        mints = psi4.core.MintsHelper(wfn.basisset())

        C = wfn.Ca() # -> as psi4 matrix object
        Cnp = np.asarray(C)

        # transform F_ao to the MO basis
        F = np.einsum('uj, vi, uv', Cnp, Cnp, F)
    
    else:
        rhf_e, wfn = psi4.energy(method, return_wfn=True)

        # get the number of orbitals and the number of doubly occupied orbitals
        n_orbitals = wfn.nmo()
        n_docc = wfn.nalpha()

        ### NOTE! the wfn object has the Fock matrix (Fa and Fb), we can 
        ### Just grab it instead of building it again.  We need to 
        ### remember that it is stored as a psi4 matrix object, so we need to
        ### turn it into a numpy array before we use it!
        
        # get the orbital transformation matrix
        C = wfn.Ca() # -> as psi4 matrix object
        Cnp = np.asarray(C) # -> as numpy array
    
    
        # instantiate the MintsHelper object
        mints = psi4.core.MintsHelper(wfn.basisset())
    
        # get the one-electron integrals
        H_ao = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())
    
        # transform H_ao to the MO basis
        H_mo = np.einsum('uj, vi, uv', Cnp, Cnp, H_ao)
    
        # get the two-electron integrals, use psi4 to transform into the MO basis because that is more efficient
        ERI =  np.asarray(mints.mo_eri(C, C, C, C))
    
        # Build the Fock matrix
        F = H_mo + 2 * np.einsum("ijkk->ij", ERI[:, :, :n_docc, :n_docc]) 
        F -= np.einsum("ikkj->ij", ERI[:, :n_docc, :n_docc, :] )

    # get number of atoms
    n_atoms = molecule.natom()

    # now compute the overlap gradient and contract with Fock matrix
    overlap_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    overlap_gradient = np.zeros(3 * n_atoms)
    for atom_index in range(n_atoms):

        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index
            # Get overlap derivatives for this atom and Cartesian component
            overlap_derivs[deriv_index, :, :] = np.asarray(mints.mo_oei_deriv1("OVERLAP", atom_index, C, C)[cart_index])
            overlap_gradient[deriv_index] = -2.0 * np.einsum('ii,ii->', F[:n_docc, :n_docc], overlap_derivs[deriv_index, :n_docc, :n_docc])

    return overlap_gradient

In [10]:
def compute_one_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf', qed=False, lambda_list=[0, 0, 0]):
    """
    NEEDS COMPLETING: Method to compute the one-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The one-electron integral gradient terms are the derivatives of the one-electron integrals with respect to the nuclear coordinates.
    To compute the one-electron integral gradient terms, we need the one-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    options_dict = {
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    }
    
    # Set up Psi4 options
    psi4.set_options(options_dict)

    # set up the geometry
    psi4.geometry(geometry_string)

    if qed:
        pqed_dict = cqed_rhf(lambda_list, geometry_string, options_dict)
        rhf_e = pqed_dict["CQED-RHF ENERGY"]
        wfn = pqed_dict["CQED-RHF WFN"]
    
    else:
        rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]

    # define density matrix as 2 * sum_i C_pi C_qi
    D = 2 * np.einsum("pi,qi->pq", Cocc, Cocc) # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the one-electron integrals derivative matrices
    kinetic_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    potential_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    

    kinetic_gradient = np.zeros(3 * n_atoms)
    potential_gradient = np.zeros(3 * n_atoms)


    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
         for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get the one-electron integral derivatives
            kinetic_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("KINETIC", atom_index)[cart_index])
            potential_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("POTENTIAL", atom_index)[cart_index])

            # add code to contract kinetic_derivs with D
            kinetic_gradient[deriv_index] = np.einsum("uv,uv->", D, kinetic_derivs[deriv_index, :, :])

            # add code to contract potential_derivs with D
            potential_gradient[deriv_index] = np.einsum("uv,uv->", D, potential_derivs[deriv_index, :, :])

    
    # add code to return the kinet_gradient and potential_gradient
    return kinetic_gradient, potential_gradient

In [11]:
# compute the overlap gradient ter
overlap_gradient = compute_fock_matrix_term(starting_string, basis_set='sto-3g', method='scf', qed=True, lambda_list=[0, 0, 0] )

# compute the one-electron integral gradient terms
kinetic_gradient, potential_gradient = compute_one_electron_integral_gradient_terms(starting_string, qed=True, lambda_list=[0, 0, 0])




Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Fri May 30 13:36:27 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------

In [12]:
# check overlap gradient against expected
print("Expected overlap gradient:\n", _overlap_gradient)
print("Computed overlap gradient:\n", overlap_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(overlap_gradient.reshape(3,3), _overlap_gradient):
    print("Overlap gradient is correct.")
else:
    print("Overlap gradient is incorrect.")

# check kinetic gradient against expected
print("Expected kinetic gradient:\n", _kinetic_gradient)
print("Computed kinetic gradient:\n", kinetic_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(kinetic_gradient.reshape(3,3), _kinetic_gradient):
    print("Kinetic gradient is correct.")
else:
    print("Kinetic gradient is incorrect.")

# check potential gradient against expected
print("Expected potential gradient:\n", _potential_gradient)
print("Computed potential gradient:\n", potential_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(potential_gradient.reshape(3,3), _potential_gradient):
    print("Potential gradient is correct.")
else:
    print("Potential gradient is incorrect.")


Expected overlap gradient:
 [[-0.               -0.                0.30728746121587]
 [ 0.               -0.149771265758   -0.15364373060793]
 [-0.                0.149771265758   -0.15364373060793]]
Computed overlap gradient:
 [[ 0.                -0.000000000000003  0.307287461215269]
 [ 0.                -0.149771265757691 -0.153643730607633]
 [-0.                 0.149771265757694 -0.153643730607636]]
Overlap gradient is correct.
Expected kinetic gradient:
 [[ 0.               -0.                0.66968290617933]
 [ 0.               -0.43735698924315 -0.33484145308966]
 [-0.                0.43735698924315 -0.33484145308967]]
Computed kinetic gradient:
 [[-0.                -0.                 0.669682906178827]
 [ 0.                -0.437356989243073 -0.334841453089413]
 [ 0.                 0.437356989243074 -0.334841453089414]]
Kinetic gradient is correct.
Expected potential gradient:
 [[-0.                0.00000000000002 -6.81982772856799]
 [-0.                4.38321774316664

# Note on the ordering of the array returned by MintsHelper ao_elec_dip_deriv1

`np.asarray(mints.ao_elec_dip_deriv1(atom_index))` returns a (9, n_orbital, n_orbital) array for a given atom index.

The inner index is ordered as follows:

\begin{align}
0 \rightarrow \frac{\partial}{\partial x} \mu^x \\
1 \rightarrow \frac{\partial}{\partial y} \mu^x \\
2 \rightarrow \frac{\partial}{\partial z} \mu^x \\
3 \rightarrow \frac{\partial}{\partial x} \mu^y \\
4 \rightarrow \frac{\partial}{\partial y} \mu^y \\
5 \rightarrow \frac{\partial}{\partial z} \mu^y \\
6 \rightarrow \frac{\partial}{\partial x} \mu^z \\
7 \rightarrow \frac{\partial}{\partial y} \mu^z \\
8 \rightarrow \frac{\partial}{\partial z} \mu^z
\end{align}

Each dipole component needs to be multiplied by the corresponding lambda component:

\begin{align}
\frac{\partial}{\partial x} d = \lambda_x \frac{\partial}{\partial x} \mu^x + \lambda_y \frac{\partial}{\partial x} \mu^y + \lambda_z \frac{\partial}{\partial x} \mu^z \\
\frac{\partial}{\partial y} d = \lambda_x \frac{\partial}{\partial y} \mu^x + \lambda_y \frac{\partial}{\partial y} \mu^y + \lambda_z \frac{\partial}{\partial y} \mu^z  \\
\frac{\partial}{\partial z} d = \lambda_x \frac{\partial}{\partial z} \mu^x + \lambda_y \frac{\partial}{\partial z} \mu^y + \lambda_z \frac{\partial}{\partial z} \mu^z 
\end{align}

These are the terms stored in `d_derivs`.

**Homework is to try to correctly compute `d_derivs`**

In [24]:
def compute_two_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf', qed=False, lambda_list=[0, 0, 0]):
    """
    NEEDS COMPLETING: Method to compute the two-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The two-electron integral gradient terms are the derivatives of the two-electron integrals with respect to the nuclear coordinates.
    To compute the two-electron integral gradient terms, we need the two-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    # Set up Psi4 options
    options_dict = {
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    }

    # add print statement for lambda_list
    print("!!!! Lambda List is !!!!!")
    print(lambda_list)
    
    # Set up Psi4 options
    psi4.set_options(options_dict)

    # set up the geometry
    psi4.geometry(geometry_string)

    if qed:
        pqed_dict = cqed_rhf(lambda_list, geometry_string, options_dict)
        rhf_e = pqed_dict["CQED-RHF ENERGY"]
        wfn = pqed_dict["CQED-RHF WFN"]
    
    else:
        rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]

    # define D = 2 * sum_i C_pi * C_qi
    D = 2 * np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the two-electron integrals derivative matrices
    eri_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals, n_orbitals, n_orbitals))
    J_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    K_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    J_gradient = np.zeros(3 * n_atoms)
    K_gradient = np.zeros(3 * n_atoms)

    # initialize three arrays for the K_dse terms
    dipole_derivs = np.zeros((3 * n_atoms, 3, n_orbitals, n_orbitals))
    d_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    d_matrix = np.zeros((n_orbitals, n_orbitals))
    K_dse_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    K_dse_gradient = np.zeros(3 * n_atoms)

    # get the dipole integral derivatives
    #dipole_derivs = np.asarray(mints.ao_elec_dip_deriv1())
    

    # get the dipole integrals
    d_matrix = lambda_list[0] * np.asarray(mints.ao_dipole()[0])
    print("x contribution of d matrix")
    print(d_matrix)
    d_matrix += lambda_list[1] * np.asarray(mints.ao_dipole()[1])
    print("y contribution of d matrix")
    print(d_matrix)
    d_matrix += lambda_list[2] * np.asarray(mints.ao_dipole()[2])
    print("z contribution of d matrix")
    print(d_matrix)

    #print("The shape of dipole_derivs is ",np.shape(dipole_derivs))
    print("The shape of d_matrix is ", np.shape(d_matrix))
                                           
    

    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get element of d_deriv
            d_derivs[deriv_index] = lambda_list[cart_index] * np.asarray(mints.ao_elec_dip_deriv1(atom_index)[cart_index])

            _temp = np.asarray(mints.ao_elec_dip_deriv1(atom_index))
            print("Printing Shape of mints.ao_elec_dip_deriv1(atom_index)")
            print(np.shape(_temp))
            print("Printing slice 0, which is d/dx mu^x")
            print(_temp[0,:,:])

            print("Printing slice 1, which is either d/dx mu^y or d/dy mu^x")
            print(_temp[1,:,:])

            print("Printing slice 2, which is either d/dx mu^z or d/dz mu^x")
            print(_temp[2,:,:])
            

            # get the two-electron integral derivatives
            eri_derivs[deriv_index] = np.asarray(mints.ao_tei_deriv1(atom_index)[cart_index])

            # add code to contract eri_derivs with D to get J_deriv. J_uv = 2 * sum_ls (uv|ls) D_ls 
            J_deriv[deriv_index] = np.einsum("uvls,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract eri_derivs with D to get K_deriv. K_uv = -1 * sum_ls (ul|vs) D_ls
            K_deriv[deriv_index] = -1 / 2 * np.einsum("ulvs,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract d_derivs * d_matrix with D to get K_deriv, K^dse_uv = -1 sum_ls * d'_us * dlv D_ls
            K_dse_deriv[deriv_index] = -1 * np.einsum("us,lv,ls->uv", d_derivs[deriv_index, :, :], d_matrix, D)

            # add code to contract J_deriv with D to get J_gradient
            J_gradient[deriv_index] = 1 / 2 * np.einsum("uv,uv->", D, J_deriv[deriv_index, :, :])

            # add code to contract K_deriv with D to get K_gradient
            K_gradient[deriv_index] = 1 / 2 * np.einsum("uv,uv->", D, K_deriv[deriv_index, :, :])

            K_dse_gradient[deriv_index] = np.einsum("uv, uv->", D, K_dse_deriv[deriv_index, :, :])

    # add code to return the J_gradient and K_gradient
    return J_gradient, K_gradient, K_dse_gradient

In [23]:
J_gradient, K_gradient, K_dse_gradient = compute_two_electron_integral_gradient_terms(starting_string, qed=True, lambda_list = [0, 0, 0.1])


!!!! Lambda List is !!!!!
[0, 0, 0.1]

Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Fri May 30 14:08:38 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -----------

In [15]:
print("J_gradient")
print(J_gradient)

print("K_gradient")
print(K_gradient)

print("K_dse_gradient")
print(K_dse_gradient)

J_gradient
[ 0.000000000000001 -0.000000000000003  3.376877640621379  0.                -2.042719762732459 -1.688438820310684 -0.000000000000001  2.042719762732462 -1.688438820310694]
K_gradient
[-0.                 0.000000000000001 -0.439854735788096 -0.                 0.270346619783886  0.219927367894047  0.                -0.270346619783887  0.219927367894049]
K_dse_gradient
[ 0.  0.  0.  0.  0.  0.  0.  0. -0.]


In [ ]:
# check J gradient against expected
print("Expected J gradient:\n", _coulomb_gradient)
print("Computed J gradient:\n", J_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(J_gradient.reshape(3,3), _coulomb_gradient):
    print("J gradient is correct.")
else:   
    print("J gradient is incorrect.")
# check K gradient against expected
print("Expected K gradient:\n", _exchange_gradient)
print("Computed K gradient:\n", K_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(K_gradient.reshape(3,3), _exchange_gradient):
    print("K gradient is correct.")
else:
    print("K gradient is incorrect.")


In [ ]:
# compute the total energy gradient
total_energy_gradient = nuclear_repulsion_gradient.flatten() + kinetic_gradient + potential_gradient + J_gradient + K_gradient + overlap_gradient
print("Total energy gradient:\n", total_energy_gradient.reshape(3,3))
# Compare the computed total energy gradient with the expected value psi4
if np.allclose(total_energy_gradient.reshape(3,3), gradient):
    print("Total energy gradient is correct.")
else:  
    print("Total energy gradient is incorrect.")
    print("Computed:\n", total_energy_gradient.reshape(3,3))
    print("Expected:\n", gradient)